<h1> Training flight delay model in BigQuery ML </h1>

Run this notebook in Cloud AI Platform Notebooks. In this notebook, we will use BigQuery ML to train the same model that we did in Spark ML.

Note how much easier this is ... (and also much more scaleable)

## Verify dataset

Let's make sure that we have the traindays and flights data in BigQuery. If you don't, please follow steps in the README.md in this directory.

In [1]:
%%bigquery
SELECT * FROM flights.tzcorr
LIMIT 5

,FL_DATE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,DEST_AIRPORT_ID,...,CANCELLED,CANCELLATION_CODE,DIVERTED,DISTANCE,DEP_AIRPORT_LAT,DEP_AIRPORT_LON,DEP_AIRPORT_TZOFFSET,ARR_AIRPORT_LAT,ARR_AIRPORT_LON,ARR_AIRPORT_TZOFFSET
0,2015-10-30,WN,19393,WN,2674,11884,1188402,31884,GEG,10713,...,0.00,None,0.00,287.0,47.619167,-117.535278,-25200.0,43.564444,-116.222778,-21600.0
1,2015-05-15,AS,19930,AS,40,10170,1017001,30070,ADQ,10299,...,0.00,None,0.00,253.0,57.750000,-152.493889,-28800.0,61.174167,-149.998056,-28800.0
2,2015-05-15,AS,19930,AS,167,10299,1029904,30299,ANC,11630,...,0.00,None,0.00,261.0,61.174167,-149.998056,-28800.0,64.815000,-147.856389,-28800.0
3,2015-05-15,B6,20409,B6,341,15027,1502704,34992,STX,14843,...,0.00,None,0.00,94.0,17.701389,-64.801944,-14400.0,18.439444,-66.002222,-14400.0
4,2015-05-16,HA,19690,HA,508,12982,1298202,32982,LIH,13830,...,0.00,None,0.00,201.0,21.976111,-159.338889,-36000.0,20.898611,-156.430556,-36000.0


In [3]:
%%bigquery
SELECT * FROM flights.trainday
LIMIT 5

,FL_DATE,is_train_day
0,2015-01-01,True
1,2015-01-02,False
2,2015-01-03,False
3,2015-01-04,True
4,2015-01-05,True


<h2> Logistic regression </h2>

This query will take about 10 minutes.

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL flights.arr_delay_lm
OPTIONS(input_label_cols=['ontime'], 
        model_type='logistic_reg', 
        data_split_method='custom',
        data_split_col='is_eval_day')
AS

SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime
  , dep_delay
  , taxi_out
  , distance
  , IF(is_train_day = 'True', False, True) AS is_eval_day
FROM flights.tzcorr f
JOIN flights.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = '0.00' AND 
  f.DIVERTED = '0.00'

Executing query with job ID: 98787503-7c80-49f7-bd88-cb1addfaf895
Query executing: 501.09s

In [17]:
%%bigquery
SELECT * 
FROM ML.EVALUATE(MODEL flights.arr_delay_lm,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime
  , dep_delay
  , taxi_out
  , distance
FROM flights.tzcorr f
JOIN flights.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = '0.00' AND 
  f.DIVERTED = '0.00' AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.962889,0.961851,0.938016,0.96237,0.166853,0.956974


## Do same metrics as in Spark code

We are using ML.PREDICT and computing the necessary stats

In [30]:
%%bigquery

WITH predictions AS (
SELECT 
  *
FROM ML.PREDICT(MODEL flights.arr_delay_lm,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime
  , dep_delay
  , taxi_out
  , distance
FROM flights.tzcorr f
JOIN flights.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = '0.00' AND 
  f.DIVERTED = '0.00' AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))),

stats AS (
SELECT 
  COUNTIF(ontime != 'ontime' AND ontime = predicted_ontime) AS correct_cancel
  , COUNTIF(predicted_ontime = 'ontime') AS total_noncancel
  , COUNTIF(ontime = 'ontime' AND ontime = predicted_ontime) AS correct_noncancel
  , COUNTIF(ontime != 'ontime') AS total_cancel
  , SQRT(SUM((IF(ontime = 'ontime', 1, 0) - p.prob) * (IF(ontime = 'ontime', 1, 0) - p.prob))/COUNT(*)) AS rmse
FROM predictions, UNNEST(predicted_ontime_probs) p
WHERE p.label = 'ontime'
)

SELECT
   correct_cancel / total_cancel AS correct_cancel
   , total_noncancel
   , correct_noncancel / total_noncancel AS correct_noncancel
   , total_cancel
   , rmse
FROM stats

,correct_cancel,total_noncancel,correct_noncancel,total_cancel,rmse
0,0.82639,1432340,0.962889,306174,0.212779


## Add airport info

Add airport information to model (note two additional columns: origin and dest). This seemingly simple change adds two categorical variables that, when one-hot-encoded, adds 600+ new columns to the model. BigQuery ML doesn't skip a beat ...

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL flights.arr_delay_airports_lm
OPTIONS(input_label_cols=['ontime'], 
        model_type='logistic_reg', 
        data_split_method='custom',
        data_split_col='is_eval_day')
AS

SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime
  , dep_delay
  , taxi_out
  , distance
  , origin
  , dest
  , IF(is_train_day = 'True', False, True) AS is_eval_day
FROM flights.tzcorr f
JOIN flights.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = '0.00' AND 
  f.DIVERTED = '0.00'

In [33]:
%%bigquery
SELECT * 
FROM ML.EVALUATE(MODEL flights.arr_delay_airports_lm,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime
  , dep_delay
  , taxi_out
  , distance
  , origin
  , dest
FROM flights.tzcorr f
JOIN flights.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = '0.00' AND 
  f.DIVERTED = '0.00' AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.967201,0.95788,0.938524,0.962518,0.165272,0.961587


In [31]:
%%bigquery

WITH predictions AS (
SELECT 
  *
FROM ML.PREDICT(MODEL flights.arr_delay_airports_lm,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime
  , dep_delay
  , taxi_out
  , origin
  , dest
  , distance
FROM flights.tzcorr f
JOIN flights.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = '0.00' AND 
  f.DIVERTED = '0.00' AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))),

stats AS (
SELECT 
  COUNTIF(ontime != 'ontime' AND ontime = predicted_ontime) AS correct_cancel
  , COUNTIF(predicted_ontime = 'ontime') AS total_noncancel
  , COUNTIF(ontime = 'ontime' AND ontime = predicted_ontime) AS correct_noncancel
  , COUNTIF(ontime != 'ontime') AS total_cancel
  , SQRT(SUM((IF(ontime = 'ontime', 1, 0) - p.prob) * (IF(ontime = 'ontime', 1, 0) - p.prob))/COUNT(*)) AS rmse
FROM predictions, UNNEST(predicted_ontime_probs) p
WHERE p.label = 'ontime'
)

SELECT
   correct_cancel / total_cancel AS correct_cancel
   , total_noncancel
   , correct_noncancel / total_noncancel AS correct_noncancel
   , total_cancel
   , rmse
FROM stats

,correct_cancel,total_noncancel,correct_noncancel,total_cancel,rmse
0,0.847874,1420068,0.967201,306174,0.209687


Note that the addition of the airports information has improved both the AUC and the RMSE

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.